# Selective flow constraints with [PowerSimulations.jl](https://github.com/NREL/PowerSimulations.jl)

**Originally Contributed by**: Clayton Barrows and Sourabh Dalvi

## Introduction

The [Operations Problems example]](https://nbviewer.jupyter.org/github/NREL-SIIP/SIIPExamples.jl/blob/master/notebook/3_PowerSimulations_examples/02_sequential_simulations.ipynb)
shows the basic building blocks of building optimization problems with PowerSimulations.jl.
This example shows how to customize the enforcement of branch flow constraints as is common
when trying to build large scale simulations.

## Dependencies
### Modeling Packages

In [1]:
using PowerSystems
using PowerSimulations
using PowerSystemCaseBuilder

### Optimization packages
For this simple example, we can use the Cbc solver with a relatively relaxed tolerance.

In [2]:
using Cbc #solver
solver = optimizer_with_attributes(Cbc.Optimizer, "logLevel" => 1, "ratioGap" => 0.5)

MathOptInterface.OptimizerWithAttributes(Cbc.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.RawParameter("logLevel") => 1, MathOptInterface.RawParameter("ratioGap") => 0.5])

### Create a `System` from RTS-GMLC data

In [3]:
sys = build_system(PSITestSystems, "test_RTS_GMLC_sys")

┌ Info: Building new system test_RTS_GMLC_sys from raw data
└   sys_descriptor.raw_data = "/home/runner/.julia/packages/PowerSystemCaseBuilder/XPh1v/data/RTS_GMLC"
[ Info: Parsing csv files in Reserves ...
[ Info: Parsing csv data in DAY_AHEAD_regional_Flex_Down.csv ...
[ Info: Parsing csv data in DAY_AHEAD_regional_Flex_Up.csv ...
[ Info: Parsing csv data in DAY_AHEAD_regional_Reg_Down.csv ...
[ Info: Parsing csv data in DAY_AHEAD_regional_Reg_Up.csv ...
[ Info: Parsing csv data in DAY_AHEAD_regional_Spin_Up_R1.csv ...
[ Info: Parsing csv data in DAY_AHEAD_regional_Spin_Up_R2.csv ...
[ Info: Parsing csv data in DAY_AHEAD_regional_Spin_Up_R3.csv ...
[ Info: Parsing csv data in REAL_TIME_regional_Reg_Down.csv ...
[ Info: Parsing csv data in REAL_TIME_regional_Reg_Up.csv ...
[ Info: Parsing csv data in REAL_TIME_regional_Spin_Up_R1.csv ...
[ Info: Parsing csv data in REAL_TIME_regional_Spin_Up_R2.csv ...
[ Info: Parsing csv data in REAL_TIME_regional_Spin_Up_R3.csv ...
[ Info: Successful

System
┌───────────────────┬─────────────┐
│ Property          │ Value       │
├───────────────────┼─────────────┤
│ System Units Base │ DEVICE_BASE │
│ Base Power        │ 100.0       │
│ Base Frequency    │ 60.0        │
│ Num Components    │ 525         │
└───────────────────┴─────────────┘

Static Components
┌───────────────────────────────────────────┬───────┬────────────────────────┬──
│ Type                                      │ Count │ Has Static Time Series │ ⋯
├───────────────────────────────────────────┼───────┼────────────────────────┼──
│ Arc                                       │ 109   │ false                  │ ⋯
│ Area                                      │ 3     │ false                  │ ⋯
│ Bus                                       │ 73    │ false                  │ ⋯
│ GenericBattery                            │ 1     │ false                  │ ⋯
│ HVDCLine                                  │ 1     │ false                  │ ⋯
│ HydroDispatch                             │ 1     │ true                   │ ⋯
│ HydroEnergyReservoir                      │ 19    │ true                   │ ⋯
│ Line                                      │ 105   │ false                  │ ⋯
│ LoadZone                                  │ 3     │ true                   │ ⋯
│ PowerLoad                                 │ 51    │ true                   │ ⋯
│ RenewableDispatch                         │ 30    │ true                   │ ⋯
│ RenewableFix                              │ 31    │ true                   │ ⋯
│ TapTransformer                            │ 15    │ false                  │ ⋯
│ ThermalStandard                           │ 76    │ false                  │ ⋯
│ VariableReserve{PowerSystems.ReserveDown} │ 2     │ true                   │ ⋯
│ VariableReserve{PowerSystems.ReserveUp}   │ 5     │ true                   │ ⋯
└───────────────────────────────────────────┴───────┴────────────────────────┴──
                                                                1 column omitted

Time Series Summary
┌──────────────────────────────────┬─────────────────────┐
│ Property                         │ Value               │
├──────────────────────────────────┼─────────────────────┤
│ Components with time series data │ 142                 │
│ Total StaticTimeSeries           │ 182                 │
│ Total Forecasts                  │ 182                 │
│ Resolution                       │ 60 minutes          │
│ First initial time               │ 2020-01-01T00:00:00 │
│ Last initial time                │ 2020-01-01T00:00:00 │
│ Horizon                          │ 24                  │
│ Interval                         │ 0 minutes           │
│ Forecast window count            │ 1                   │
└──────────────────────────────────┴─────────────────────┘

### Selecting flow limited lines
Since PowerSimulations will apply constraints by component type (e.g. Line), we need to
change the component type of the lines on which we want to enforce flow limits. So, let's
change the device type of certain branches from Line to MonitoredLine differentiate
treatment when we build the model. Here, we can select inter-regional lines, or lines
above a voltage threshold.

In [4]:
for line in get_components(Line, sys)
    if (get_base_voltage(get_from(get_arc(line))) >= 230.0) &&
       (get_base_voltage(get_to(get_arc(line))) >= 230.0)
        #if get_area(get_from(get_arc(line))) != get_area(get_to(get_arc(line)))
        @info "Changing $(get_name(line)) to MonitoredLine"
        convert_component!(MonitoredLine, line, sys)
    end
end

[ Info: Changing C31-2 to MonitoredLine
[ Info: Changing B26 to MonitoredLine
[ Info: Changing A32-2 to MonitoredLine
[ Info: Changing CA-1 to MonitoredLine
[ Info: Changing B34 to MonitoredLine
[ Info: Changing B19 to MonitoredLine
[ Info: Changing B31-2 to MonitoredLine
[ Info: Changing C30 to MonitoredLine
[ Info: Changing B27 to MonitoredLine
[ Info: Changing A21 to MonitoredLine
[ Info: Changing A32-1 to MonitoredLine
[ Info: Changing B29 to MonitoredLine
[ Info: Changing A18 to MonitoredLine
[ Info: Changing C24 to MonitoredLine
[ Info: Changing C28 to MonitoredLine
[ Info: Changing A29 to MonitoredLine
[ Info: Changing AB2 to MonitoredLine
[ Info: Changing C27 to MonitoredLine
[ Info: Changing C35 to MonitoredLine
[ Info: Changing A25-1 to MonitoredLine
[ Info: Changing B31-1 to MonitoredLine
[ Info: Changing C20 to MonitoredLine
[ Info: Changing B23 to MonitoredLine
[ Info: Changing A28 to MonitoredLine
[ Info: Changing A19 to MonitoredLine
[ Info: Changing B25-1 to MonitoredLi

Let's start with a standard unit commitment template using the `PTDFPowerModel` network
formulation which only constructs the admittance matrix rows corresponding to "bounded" lines:

In [5]:
template = template_unit_commitment(transmission = PTDFPowerModel)


Operations Problem Specification
Transmission: PowerSimulations.CopperPlatePowerModel
Devices Models: 

	Type: PowerSystems.ThermalStandard
 	Formulation: PowerSimulations.ThermalBasicUnitCommitment

	Type: PowerSystems.HydroEnergyReservoir
 	Formulation: PowerSimulations.HydroDispatchRunOfRiver

	Type: PowerSystems.RenewableDispatch
 	Formulation: PowerSimulations.RenewableFullDispatch

	Type: PowerSystems.PowerLoad
 	Formulation: PowerSimulations.StaticPowerLoad

	Type: PowerSystems.InterruptibleLoad
 	Formulation: PowerSimulations.InterruptiblePowerLoad

	Type: PowerSystems.RenewableFix
 	Formulation: PowerSimulations.FixedOutput

	Type: PowerSystems.HydroDispatch
 	Formulation: PowerSimulations.HydroDispatchRunOfRiver

Branches Models: 

	Type: PowerSystems.Transformer2W
 	Formulation: PowerSimulations.StaticBranch

	Type: PowerSystems.Line
 	Formulation: PowerSimulations.StaticBranch

	Type: PowerSystems.HVDCLine
 	Formulation: PowerSimulations.HVDCDispatch

	Type: PowerSystems.T

Notice that there is no entry for `MonitoredLine`, so we can add one:

In [6]:
set_device_model!(template, MonitoredLine, StaticBranch)

We can also relax the formulation applied to the `Line` components to an unbounded flow formulation.
This formulation still enforces Kirchoff's laws, but does not apply flow constraints.

In [7]:
set_device_model!(template, Line, StaticBranchUnbounded)

[ Info: Overwriting PowerSystems.Line existing model


## Build an `OperationsProblem`

In [8]:
uc_prob = OperationsProblem(template, sys, horizon = 24, optimizer = solver)
build!(uc_prob, output_dir = mktempdir())

PowerSimulations.BuildStatusModule.BuildStatus.BUILT = 0

Solve the relaxed problem

In [9]:
solve!(uc_prob)

PowerSimulations.RunStatusModule.RunStatus.SUCCESSFUL = 0

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*